## Chapter 20: Caching for Performance

In modern web applications, performance is paramount. Users expect pages to load quickly, and APIs to respond in milliseconds. One of the most effective ways to improve performance is **caching**—storing frequently accessed data in a fast storage layer so that subsequent requests can be served without repeating expensive operations like database queries or external API calls. ASP.NET Core provides a flexible caching infrastructure that includes in‑memory caching, distributed caching, and response caching. In this chapter, you’ll learn how to use these caching techniques, understand when to apply each, and master cache invalidation strategies to ensure your data remains fresh. By the end, you’ll be able to dramatically reduce latency and database load in your applications.

### 20.1 In‑Memory Caching (`IMemoryCache`)

In‑memory caching stores data in the memory of the web server. It’s the simplest and fastest form of caching, ideal for single‑server deployments or when cached data doesn’t need to be shared across multiple servers.

#### Adding In‑Memory Cache

In‑memory caching services are added by calling `AddMemoryCache()` in `Program.cs`:

```csharp
builder.Services.AddMemoryCache();
```

Then you can inject `IMemoryCache` into any controller or service.

#### Basic Usage

```csharp
using Microsoft.Extensions.Caching.Memory;

public class ProductsController : ControllerBase
{
    private readonly IMemoryCache _cache;
    private readonly IProductRepository _repository;

    public ProductsController(IMemoryCache cache, IProductRepository repository)
    {
        _cache = cache;
        _repository = repository;
    }

    [HttpGet("{id}")]
    public async Task<ActionResult<Product>> GetProduct(int id)
    {
        string cacheKey = $"product_{id}";

        // Try to get the product from cache
        if (!_cache.TryGetValue(cacheKey, out Product product))
        {
            // Not in cache, retrieve from repository
            product = await _repository.GetByIdAsync(id);
            if (product == null)
                return NotFound();

            // Store in cache with options
            _cache.Set(cacheKey, product, new MemoryCacheEntryOptions
            {
                AbsoluteExpirationRelativeToNow = TimeSpan.FromMinutes(5),
                SlidingExpiration = TimeSpan.FromMinutes(2)
            });
        }

        return Ok(product);
    }
}
```

#### Cache Entry Options

- **AbsoluteExpirationRelativeToNow**: The cache entry will expire after a fixed duration, regardless of how often it’s accessed.
- **SlidingExpiration**: The cache entry expires if it hasn’t been accessed for the specified duration. Each access resets the expiration timer.
- **Priority**: Allows you to set a priority (Low, Normal, High, NeverRemove) that influences cache eviction when memory pressure is high.
- **Size**: For cache size limits (if configured), you can assign a size to each entry.

You can combine both absolute and sliding expiration. The entry expires when either condition is met.

#### Removing Items from Cache

You can manually remove an item:

```csharp
_cache.Remove(cacheKey);
```

#### When to Use In‑Memory Cache

- Data that is expensive to create or retrieve.
- Data that changes infrequently.
- Single‑server applications (not suitable for web farms unless using sticky sessions or a distributed cache).

### 20.2 Distributed Caching (Redis)

When your application runs on multiple servers, in‑memory caches become inconsistent because each server has its own copy. A **distributed cache** shares cached data across all servers, providing a single source of truth. **Redis** is the most popular distributed cache; it’s an in‑memory data store that supports high throughput and low latency.

#### Setting Up Redis

First, install the Redis cache provider:

```bash
dotnet add package Microsoft.Extensions.Caching.StackExchangeRedis
```

Then, in `Program.cs`, add the distributed Redis cache:

```csharp
builder.Services.AddStackExchangeRedisCache(options =>
{
    options.Configuration = builder.Configuration.GetConnectionString("Redis");
    options.InstanceName = "MyApp:"; // optional prefix
});
```

The connection string might look like: `localhost:6379` or a full Azure Redis connection string.

#### Using `IDistributedCache`

The `IDistributedCache` interface provides methods for working with byte arrays, strings, and objects. It’s intentionally low‑level; you’ll often serialize your objects to JSON or binary.

```csharp
using Microsoft.Extensions.Caching.Distributed;
using System.Text.Json;

public class ProductService
{
    private readonly IDistributedCache _cache;
    private readonly IProductRepository _repository;

    public ProductService(IDistributedCache cache, IProductRepository repository)
    {
        _cache = cache;
        _repository = repository;
    }

    public async Task<Product> GetProductAsync(int id)
    {
        string cacheKey = $"product_{id}";
        string cachedJson = await _cache.GetStringAsync(cacheKey);

        if (!string.IsNullOrEmpty(cachedJson))
        {
            return JsonSerializer.Deserialize<Product>(cachedJson);
        }

        var product = await _repository.GetByIdAsync(id);
        if (product != null)
        {
            var options = new DistributedCacheEntryOptions
            {
                AbsoluteExpirationRelativeToNow = TimeSpan.FromMinutes(5)
            };
            await _cache.SetStringAsync(cacheKey, JsonSerializer.Serialize(product), options);
        }
        return product;
    }
}
```

#### Distributed Cache Entry Options

- **AbsoluteExpiration**: an absolute `DateTimeOffset`.
- **AbsoluteExpirationRelativeToNow**: a `TimeSpan` from now.
- **SlidingExpiration**: a `TimeSpan` of inactivity before expiration.

Note that not all distributed cache implementations support sliding expiration; Redis does.

#### When to Use Distributed Cache

- When your application runs on multiple servers.
- When cached data needs to be shared.
- When you need a central cache that survives server restarts (if Redis is configured with persistence).

#### Redis Beyond Caching

Redis can also be used for pub/sub messaging, rate limiting, and as a data structure server. Many ASP.NET Core applications use Redis for both caching and SignalR backplane.

### 20.3 Response Caching Middleware

**Response caching** allows the server to set HTTP headers that instruct the client (browser) or intermediate proxies to cache the response. This reduces server load and improves perceived performance because the browser doesn’t even send a request for cached resources.

ASP.NET Core provides a Response Caching Middleware that adds the appropriate headers and can also cache responses on the server (though server‑side response caching is limited; most often it’s used to set client‑side headers).

#### Adding Response Caching Middleware

```csharp
builder.Services.AddResponseCaching();

// ... later in the pipeline
app.UseResponseCaching(); // Place after static files, before endpoints
```

#### Using `[ResponseCache]` Attribute

Apply the `[ResponseCache]` attribute to controllers or actions to control caching behavior.

```csharp
[ApiController]
[Route("api/[controller]")]
public class ProductsController : ControllerBase
{
    [HttpGet]
    [ResponseCache(Duration = 60, Location = ResponseCacheLocation.Any, NoStore = false)]
    public IActionResult GetProducts()
    {
        // ...
    }

    [HttpGet("{id}")]
    [ResponseCache(Duration = 300, VaryByQueryKeys = new[] { "id" })]
    public IActionResult GetProduct(int id)
    {
        // ...
    }
}
```

- **Duration**: Cache lifetime in seconds.
- **Location**: Where the response can be cached (`Any`, `Client`, `None`). `Any` allows both client and proxy caches.
- **NoStore**: If true, sets `Cache-Control: no-store`.
- **VaryByQueryKeys**: Varies the cache by the specified query keys.
- **VaryByHeader**: Varies by header (e.g., `Accept-Encoding`).

The middleware adds headers like `Cache-Control: public,max-age=60`. The client (browser) will then cache the response for 60 seconds.

#### Server‑Side Response Caching

The response caching middleware can also store responses on the server (in‑memory) and serve them for subsequent requests, avoiding controller execution entirely. To enable server‑side caching, you must configure it and ensure the response meets the criteria (e.g., status code 200, method GET or HEAD, no `Authorization` header).

```csharp
app.UseResponseCaching();
```

When enabled, the middleware will cache responses based on the cache control headers. This can be very effective for public, read‑only endpoints.

#### Important Considerations

- Response caching is **not** suitable for authenticated requests because the `Authorization` header prevents caching.
- Use it for public data that changes infrequently.
- Be careful with `VaryByQueryKeys` to avoid serving incorrect data.

### 20.4 Cache Invalidation Strategies

Caching is powerful, but it introduces the challenge of **stale data**. When the underlying data changes, you must invalidate (remove or update) the cache to ensure users see fresh information. Invalidation is often the hardest part of caching.

#### Time‑Based Expiration (TTL)

The simplest strategy: set an expiration time. The cache automatically discards entries after a fixed duration. This works well for data that changes predictably (e.g., a list of products that updates hourly).

```csharp
_cache.Set(cacheKey, data, TimeSpan.FromMinutes(10));
```

#### Manual Invalidation

When you know exactly when data changes, you can explicitly remove or update the cache.

```csharp
public async Task<Product> UpdateProduct(Product product)
{
    // Update database
    await _repository.UpdateAsync(product);

    // Invalidate cache for this product
    string cacheKey = $"product_{product.Id}";
    _cache.Remove(cacheKey);

    // Also invalidate list caches if needed
    _cache.Remove("products_all");

    return product;
}
```

This ensures that the next read fetches fresh data from the database and repopulates the cache.

#### Tag‑Based Invalidation

Some caching systems (like Redis with custom implementation) support tagging: you assign tags to cache entries and invalidate all entries with a given tag. This is useful when a change affects many cached items (e.g., updating a category affects all products in that category).

ASP.NET Core’s built‑in caches don’t support tagging, but you can simulate it by storing keys in a separate cache entry. For example:

```csharp
// When caching a product, add its ID to a set of keys for "category:electronics"
var categoryTag = $"category:{product.CategoryId}";
var tagSetKey = $"tag:{categoryTag}";
var tagSet = await _cache.GetAsync<List<string>>(tagSetKey) ?? new List<string>();
tagSet.Add(cacheKey);
_cache.Set(tagSetKey, tagSet, TimeSpan.FromHours(1));
_cache.Set(cacheKey, product, TimeSpan.FromHours(1));

// To invalidate category:
var keys = await _cache.GetAsync<List<string>>($"tag:category:{categoryId}");
if (keys != null)
{
    foreach (var key in keys)
        _cache.Remove(key);
    _cache.Remove($"tag:category:{categoryId}");
}
```

This approach is manual but works.

#### Cache‑Aside Pattern

The pattern we’ve been using (check cache, if miss load from source and store) is called **cache‑aside** (or lazy loading). It’s simple and effective. When combined with manual invalidation on writes, it’s a solid strategy.

#### Write‑Through Cache

In a write‑through cache, writes go to the cache first, which then writes to the database. This keeps the cache consistent but adds complexity. Not commonly used with `IMemoryCache` but possible.

### 20.5 Caching in Distributed Systems

When you have multiple services, you might need a distributed cache like Redis. The same patterns apply, but you must consider serialization and network latency. Also, be mindful of **cache stampede**—when many requests simultaneously miss the cache and all hit the database. Solutions include:

- **Locks**: Only one request populates the cache; others wait (e.g., using `SemaphoreSlim`).
- **Early expiration**: Refresh the cache before it expires (e.g., if a cache entry is near its TTL and still frequently accessed, a background job can refresh it).

### 20.6 Performance Monitoring and Cache Hit Ratios

To know if your caching is effective, monitor the **cache hit ratio** (percentage of requests served from cache). You can implement simple counters:

```csharp
public class ProductService
{
    private readonly IMemoryCache _cache;
    private readonly IProductRepository _repository;
    private int _hits, _misses;

    public async Task<Product> GetProductAsync(int id)
    {
        string key = $"product_{id}";
        if (_cache.TryGetValue(key, out Product product))
        {
            Interlocked.Increment(ref _hits);
            return product;
        }

        Interlocked.Increment(ref _misses);
        product = await _repository.GetByIdAsync(id);
        if (product != null)
            _cache.Set(key, product, TimeSpan.FromMinutes(5));
        return product;
    }

    public double GetHitRatio() => _hits + _misses == 0 ? 0 : (double)_hits / (_hits + _misses);
}
```

You can expose this via a metrics endpoint or log it periodically.

### 20.7 Caching Best Practices

1. **Cache only when necessary** – Not everything benefits from caching. Profile first, then optimize.
2. **Choose the right cache type** – In‑memory for single server, distributed for multi‑server.
3. **Set appropriate expiration** – Too long risks stale data, too short reduces benefit.
4. **Invalidate proactively** – When data changes, remove or update cache entries.
5. **Be careful with caching user‑specific data** – It can accidentally leak data between users if keys are not properly scoped (include user ID in cache key).
6. **Avoid caching large objects** – They consume memory and increase serialization cost. Cache only what you need.
7. **Use `CancellationToken` with async caching** – When implementing custom cache logic, respect cancellation.
8. **Consider cache dependencies** – For example, in `IMemoryCache`, you can use `ChangeToken` to expire cache entries when a file changes (advanced).

### 20.8 Putting It All Together: Caching a Product Catalog

Let’s build a complete example: a product catalog with a list endpoint and detail endpoint, using a distributed cache (Redis) for the list and in‑memory for details (if single server, but for demonstration we’ll use distributed for both).

**appsettings.json**:
```json
{
  "ConnectionStrings": {
    "Redis": "localhost:6379"
  }
}
```

**Program.cs**:
```csharp
builder.Services.AddStackExchangeRedisCache(options =>
{
    options.Configuration = builder.Configuration.GetConnectionString("Redis");
});
builder.Services.AddMemoryCache(); // optional, for second‑level cache
builder.Services.AddScoped<IProductRepository, ProductRepository>();
builder.Services.AddScoped<ProductService>();
```

**ProductService.cs**:
```csharp
public class ProductService
{
    private readonly IDistributedCache _distributedCache;
    private readonly IMemoryCache _memoryCache;
    private readonly IProductRepository _repository;
    private readonly ILogger<ProductService> _logger;

    public ProductService(IDistributedCache distributedCache, IMemoryCache memoryCache, IProductRepository repository, ILogger<ProductService> logger)
    {
        _distributedCache = distributedCache;
        _memoryCache = memoryCache;
        _repository = repository;
        _logger = logger;
    }

    public async Task<IEnumerable<ProductDto>> GetAllProductsAsync()
    {
        const string cacheKey = "products_all";
        // Try in‑memory first (very fast)
        if (_memoryCache.TryGetValue(cacheKey, out IEnumerable<ProductDto> cachedList))
        {
            _logger.LogDebug("Products list retrieved from in‑memory cache");
            return cachedList;
        }

        // Try distributed cache
        byte[] cachedBytes = await _distributedCache.GetAsync(cacheKey);
        if (cachedBytes != null)
        {
            var list = DeserializeFromBytes<IEnumerable<ProductDto>>(cachedBytes);
            // Store in in‑memory for next time
            _memoryCache.Set(cacheKey, list, TimeSpan.FromMinutes(2)); // short sliding
            _logger.LogDebug("Products list retrieved from distributed cache");
            return list;
        }

        // Fallback to repository
        var products = await _repository.GetAllAsync();
        var dtos = products.Select(p => MapToDto(p)).ToList();

        // Store in distributed cache (longer TTL)
        var options = new DistributedCacheEntryOptions
        {
            AbsoluteExpirationRelativeToNow = TimeSpan.FromMinutes(10)
        };
        await _distributedCache.SetAsync(cacheKey, SerializeToBytes(dtos), options);

        // Also store in in‑memory for quick access
        _memoryCache.Set(cacheKey, dtos, TimeSpan.FromMinutes(2));

        _logger.LogDebug("Products list retrieved from database");
        return dtos;
    }

    public async Task<ProductDto> GetProductByIdAsync(int id)
    {
        string cacheKey = $"product_{id}";
        if (_memoryCache.TryGetValue(cacheKey, out ProductDto cachedProduct))
        {
            return cachedProduct;
        }

        byte[] cachedBytes = await _distributedCache.GetAsync(cacheKey);
        if (cachedBytes != null)
        {
            var product = DeserializeFromBytes<ProductDto>(cachedBytes);
            _memoryCache.Set(cacheKey, product, TimeSpan.FromMinutes(2));
            return product;
        }

        var entity = await _repository.GetByIdAsync(id);
        if (entity == null) return null;

        var dto = MapToDto(entity);
        var options = new DistributedCacheEntryOptions
        {
            AbsoluteExpirationRelativeToNow = TimeSpan.FromMinutes(10)
        };
        await _distributedCache.SetAsync(cacheKey, SerializeToBytes(dto), options);
        _memoryCache.Set(cacheKey, dto, TimeSpan.FromMinutes(2));
        return dto;
    }

    public async Task UpdateProductAsync(ProductDto productDto)
    {
        // Update database
        await _repository.UpdateAsync(MapToEntity(productDto));

        // Invalidate caches
        string productKey = $"product_{productDto.Id}";
        const string allKey = "products_all";

        await _distributedCache.RemoveAsync(productKey);
        await _distributedCache.RemoveAsync(allKey);
        _memoryCache.Remove(productKey);
        _memoryCache.Remove(allKey);
    }

    // Helper methods for serialization
    private byte[] SerializeToBytes<T>(T obj) => Encoding.UTF8.GetBytes(JsonSerializer.Serialize(obj));
    private T DeserializeFromBytes<T>(byte[] bytes) => JsonSerializer.Deserialize<T>(Encoding.UTF8.GetString(bytes));
}
```

In a controller, inject `ProductService` and use its methods. This example demonstrates a multi‑level cache: in‑memory (fastest) backed by distributed cache (shared) backed by database. When an update occurs, both caches are invalidated.

### Summary

In this chapter, you’ve learned the essential caching techniques for ASP.NET Core:

- **In‑memory caching** with `IMemoryCache` for single‑server, fast access.
- **Distributed caching** with Redis for multi‑server environments and shared data.
- **Response caching** to set HTTP headers and optionally cache responses on the server.
- **Cache invalidation** strategies including TTL, manual removal, and tag‑based invalidation.
- **Best practices** to avoid stale data, cache stampede, and excessive memory usage.

With these tools, you can dramatically improve your application’s performance and scalability.

**Exercise:**

1. Add in‑memory caching to an existing read‑only endpoint in your application. Measure response time before and after.
2. Set up Redis (using Docker: `docker run -p 6379:6379 redis`) and modify your application to use `IDistributedCache` with Redis for a frequently accessed endpoint.
3. Implement a simple counter to track cache hit ratio and log it.
4. Add response caching to a public API endpoint and verify the `Cache-Control` header using browser dev tools.
5. In your product update action, implement cache invalidation for both the individual product and the list of products.

In the next chapter, **"ASP.NET Core with a Modern JavaScript Frontend,"** you’ll learn how to integrate ASP.NET Core with frontend frameworks like React, Angular, or Vue, and how to structure your application as a backend API serving a single‑page application (SPA).